In [ ]:
# ADDED:
model_path = "meta-llama/Llama-2-7b-chat-hf" # not enough RAM or disk space for larger models

In [ ]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer,AutoModelForCausalLM

# change device map from "auto" to None when using CPU
tokenizer = AutoTokenizer.from_pretrained(model_path, device_map=None)

# ADDED:
torch.mps.empty_cache()
torch.cuda.empty_cache()
#

# change device map from "auto" to None when using CPU
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=None)

# ADDED:
torch.mps.empty_cache()
torch.cuda.empty_cache()

# move to CPU (not enough GPU space)
device = torch.device("cpu")
model.to(device)
#

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/proj/vondrick4/hc3295/cuda118_new/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/proj/vondrick4/hc3295/cuda118_new/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from selfie.interpret import InterpretationPrompt, interpret
interpretation_prompt = InterpretationPrompt(tokenizer, ("[INST]", 0, 0, 0, 0, 0, "[/INST] Sure, I will summarize the message:"))

# ADDED:
torch.mps.empty_cache()
torch.cuda.empty_cache()
#

In [ ]:
original_prompt = "[INST] What's highest mountain in the world? [/INST]"
tokens_to_interpret = [(10,5), (10,6)]
bs = 16 #originally: 2
max_new_tokens = 10
k = 15 #originally: 1

interpretation_df = interpret(original_prompt=original_prompt, tokens_to_interpret=tokens_to_interpret, model=model, interpretation_prompt=interpretation_prompt, bs=bs, max_new_tokens=max_new_tokens, k=k, tokenizer=tokenizer)

Interpreting '[INST] What's highest mountain in the world? [/INST]' with '[INST]_ _ _ _ _ [/INST] Sure, I will summarize the message:'


  0%|          | 0/1 [00:00<?, ?it/s]/proj/vondrick4/hc3295/selfie-dist/selfie/generate_wrappers.py:733: UserWarning: You are calling transformers.generation.generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


In [7]:
pd.DataFrame(interpretation_df)

,prompt,interpretation,layer,token,token_decoded
0,[INST] What's highest mountain in the world? [...,\n\nThe message is about the importance of taking,10,5,'
1,[INST] What's highest mountain in the world? [...,\n\nThe user is asking for a summary of,10,6,s


In [ ]:
interpretation_df